#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# ... if needed
# !jupyter nbextension enable --py gmaps       

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Load the csv exported in Part I to a DataFrame

import_data_file = "../Resources/cities.csv"
imported_df = pd.read_csv(import_data_file, dtype = "object", encoding = "utf-8")
imported_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.5,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.7,78.8,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.2,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.6,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.6,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.2,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.0,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps, set locations (lat, lng) and humidity levels (humidity)

gmaps.configure(api_key = g_key)
locations = imported_df[["Lat", "Lng"]].astype(float)
humidity = imported_df[["Humidity"]].unstack().astype(float)

In [5]:
# Configure gmaps.figure

fig = gmaps.figure(center = (0,0), zoom_level = 2) 

# Create Heatmap with Locations and Humidity as the Weight
heatmap = gmaps.heatmap_layer(locations, weights = humidity,  
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 2)

#Add Heatmap layer to map
fig.add_layer(heatmap)


#Display figure and save as .png
plt.savefig("../Images/humidity.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create new DataFrame with cities fitting weather criteria:
# Max temperature lower than 87 degrees but higher than 67
# Humidity less than 60%
# Cloudiness less than 5%

ideal_conditions_df = imported_df.loc[(imported_df["Max Temp"].astype(float) > 67) &
                                     (imported_df["Max Temp"].astype(float) < 87) &
                                     (imported_df["Humidity"].astype(float) < 60) &
                                     (imported_df["Cloudiness"].astype(float) < 5),:]

ideal_conditions_df = ideal_conditions_df.reset_index()
ideal_conditions_df.count()

index         20
City_ID       20
City          20
Cloudiness    20
Country       18
Date          20
Humidity      20
Lat           20
Lng           20
Max Temp      20
Wind Speed    20
dtype: int64

In [7]:
ideal_conditions_df.dropna(inplace = True)
#ideal_conditions_df.isnull().values.any()
#ideal_conditions_df.dropna

In [8]:
ideal_conditions_df

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,56,56,pisco,0,PE,1558378764,30,-13.71,-76.2,75.99,11.41
3,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.0,73.68,5.32
4,142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.0,3.36
5,168,168,straumen,0,NO,1558378786,52,63.87,11.3,68.0,9.17
6,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.2,6.93
7,214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.0,6.93
8,230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.8,13.87
9,244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
10,259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.2
11,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77


### Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create new DataFrame that will contain hotel info
hotel_df = pd.DataFrame([]) 

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iterate through potential ideal weather cities
# Get information for hotels within 5000m of the starting location

for index, row in ideal_conditions_df.iterrows():

    location = f"{row['Lat']}, {row['Lng']}"
    
    #set parameters for the Google Places Nearby search
    
    params = {
    "radius": 5000,
    "location": location,
    "type": "lodging",
    "keyword": "Hotel",
    "key": g_key,
    }

    
# Establish url and make API request
    print(f"Retrieving Results for {index}: {row['City']}.")
    response = requests.get(base_url, params = params).json()
   
    # extract results
    results = response['results']
    
# Populate hotel_df with hotel information
# If a hotel doesn't exist, skip the location
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'City'] = ideal_conditions_df.iloc[index, 2]
        hotel_df.loc[index, 'Country'] = ideal_conditions_df.iloc[index, 4]
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']

        
    except (KeyError, IndexError):
        print("---------------------------------------------")
        print("Missing data... Let's skip this location...")
        print("---------------------------------------------")

        pass

print("---------------------------------------------")    
print("Completed / Finished / End of Count")
print("---------------------------------------------")

Retrieving Results for 2: pisco.
Closest hotel is Embassy Beach.
Retrieving Results for 3: ponta do sol.
Closest hotel is Kanto Island Inn Suites & Spa.
Retrieving Results for 4: sarai mir.
Closest hotel is Tube well mirza rehan.
Retrieving Results for 5: straumen.
Closest hotel is Jægtvolden Fjordhotell.
Retrieving Results for 6: nador.
Closest hotel is Hotel Marchica Lagoon Resort, Nador Morocco.
Retrieving Results for 7: mount isa.
Closest hotel is Abacus Motel.
Retrieving Results for 8: moses lake.
Closest hotel is Holiday Inn Express & Suites Moses Lake.
Retrieving Results for 9: pasni.
Closest hotel is 3G hotel pasni.
Retrieving Results for 10: hami.
Closest hotel is Hami Hotel.
Retrieving Results for 11: mogok.
Closest hotel is Mount Mogok Hotel.
Retrieving Results for 12: port shepstone.
Closest hotel is The Tweni Waterfront Guest Lodge.
Retrieving Results for 13: pampierstad.
---------------------------------------------
Missing data... Let's skip this location...
------------

In [10]:
# Print hotel list
hotel_df 

,Hotel Name,City,Country,Lat,Lng
2,Embassy Beach,sarai mir,IN,-13.709870,-76.214602
3,Kanto Island Inn Suites & Spa,straumen,NO,-20.633545,-46.030340
4,Tube well mirza rehan,nador,MA,26.047660,82.961708
5,Jægtvolden Fjordhotell,mount isa,AU,63.865047,11.257339
6,"Hotel Marchica Lagoon Resort, Nador Morocco",moses lake,US,35.206750,-2.912102
7,Abacus Motel,pasni,PK,-20.699471,139.486778
8,Holiday Inn Express & Suites Moses Lake,hami,CN,47.103563,-119.252165
9,3G hotel pasni,mogok,MM,25.263623,63.470265
10,Hami Hotel,port shepstone,ZA,42.816837,93.520449
11,Mount Mogok Hotel,pampierstad,ZA,22.909694,96.503862


In [11]:
# Drop NaN again
hotel_df.dropna(inplace=True)

In [12]:
# Print new hotel db
hotel_df

,Hotel Name,City,Country,Lat,Lng
2,Embassy Beach,sarai mir,IN,-13.709870,-76.214602
3,Kanto Island Inn Suites & Spa,straumen,NO,-20.633545,-46.030340
4,Tube well mirza rehan,nador,MA,26.047660,82.961708
5,Jægtvolden Fjordhotell,mount isa,AU,63.865047,11.257339
6,"Hotel Marchica Lagoon Resort, Nador Morocco",moses lake,US,35.206750,-2.912102
7,Abacus Motel,pasni,PK,-20.699471,139.486778
8,Holiday Inn Express & Suites Moses Lake,hami,CN,47.103563,-119.252165
9,3G hotel pasni,mogok,MM,25.263623,63.470265
10,Hami Hotel,port shepstone,ZA,42.816837,93.520449
11,Mount Mogok Hotel,pampierstad,ZA,22.909694,96.503862


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# # Create hotel layer
# hotel_layer = gmaps.symbol_layer(locations, fill_color = 'black', scale = 2,
#     #stroke_color='blue', scale=1,
#     info_box_content=hotel_info
# )

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display Map and save .png
plt.savefig("../Images/hotels.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>